In [1]:
# %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import subprocess
import scipy as sc
from itertools import product

mpl.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "axes.labelsize": 12,
    "font.size": 12,
    "legend.fontsize": 10,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "figure.figsize": (4, 3.5),
    "text.latex.preamble": "\n".join([
        r"\usepackage[utf8]{inputenc}",
        r"\usepackage[T1]{fontenc}",
        r"\usepackage[detect-all,locale=FR]{siunitx}",
    ]),
    'lines.markersize': 10,
    'lines.color': 'grey',
    'scatter.marker': '+',
    'errorbar.capsize': 3,
    'savefig.bbox': 'tight',
})

In [2]:
path = '../'
executable = 'bin/ex2'
export_path = path + 'rapport/figures/'
data_path = 'data/'
config_path = 'bin/'
config_name = 'poincarre1'
config_ext = '.conf'

In [3]:
def stringify_dict(d: dict, sep=',') -> str:
    return sep.join(map(lambda a: str(a[0]) + "=" + str(a[1]), tuple(d.items())))

In [4]:
def run(config_file: str, output_file: str, params: dict = {}) -> None:
    # print(f"Running with config file {config_file}")
    options = stringify_dict(params, sep=' ')
    cmd = f"{path}{executable} {path}{config_file} output='{path}{output_file}' {options}"
    print(f"Running command `{cmd}`")
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL)
    # print("Done")

In [5]:
def load_conf() -> dict:
    conf = {}
    with open(path + config_path + config_name + config_ext, 'r') as f:
        lines = f.read().split('\n')

    for line in lines:
        if not line:
            continue
        name, _, val, *_ = line.split(' ')
        name = name.strip()
        val = val.strip()
        try:
            conf[name] = float(val)
        except ValueError:
            conf[name] = val
    
    return conf

In [6]:
conf = load_conf()

omega0 = np.sqrt(conf['g']/conf['L'])
2*omega0

14.007141035914502

In [7]:
%%bash
cd .. && make clean && make

clang++ -o bin/ex2 src/Ex2_2024_student.cpp -std=c++17 -Wall -Wextra -O2 


In [26]:
# all_params = (
#     *({'theta0': i, 'thetadot0': 0} for i in np.linspace(1, np.pi, 4)),
#     *({'theta0': 0, 'thetadot0': i} for i in np.linspace(1, 5, 4))
# )
all_params = (
    # *({'theta0': i, 'thetadot0': 1} for i in [0.0001, 1]),
    {'theta0': np.pi/6, 'thetadot0': 0},
    {'theta0': np.pi/6, 'thetadot0': 1},
    {'theta0': np.pi + 1e-6, 'thetadot0': 1e-6},
    {'theta0': np.pi + 1e-6, 'thetadot0': 0},
)

# Run simulations
outputs = []  # List to store output file names
for params in all_params:
    output_file = f"{data_path}{config_name},{stringify_dict(params)}.out"
    outputs.append(output_file)
    run(f'{config_path}{config_name}{config_ext}', output_file, params)

Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=0.5235987755982988,thetadot0=0.out' theta0=0.5235987755982988 thetadot0=0`
Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=0.5235987755982988,thetadot0=1.out' theta0=0.5235987755982988 thetadot0=1`
Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=3.1415936535897933,thetadot0=1e-06.out' theta0=3.1415936535897933 thetadot0=1e-06`
Running command `../bin/ex2 ../bin/poincarre1.conf output='../data/poincarre1,theta0=3.1415936535897933,thetadot0=0.out' theta0=3.1415936535897933 thetadot0=0`


In [27]:
# Load stuff in memory
all_data = {}

for file, params in zip(outputs, all_params):
    data = np.loadtxt(path + file)
    all_data[file] = (params, np.copy(data))

# Plots!

In [31]:
%matplotlib qt
plt.close()
for (params, data) in all_data.values():
    theta, thetadot = data[:,1], data[:,2]
    plt.scatter(theta, thetadot, s=1, marker='.')
plt.xlabel(r'$\theta$ [rad]')
plt.ylabel(r'$\dot{\theta}$ [rad/s]')
plt.grid()
# plt.show()